In [10]:
from sqlalchemy import func, desc
from datetime import date, timedelta
from sqlalchemy.orm import joinedload
from src.models.articles import Article
from src.models.stores import Store
from src.models.prices import Price
from src.database_config.database import engine

from sqlalchemy.orm import Session

session = Session(engine)

# last 30 days
start_date = date.today() - timedelta(days=30)

# query prices joined with articles and stores
query = (
    session.query(
        Store.name.label("store_name"),
        Article.name.label("article_name"),
        Price.price,
        func.avg(Price.price).over(partition_by=Store.id).label("avg_store_price")
    )
    .join(Article, Article.id == Price.article_id)
    .join(Store, Store.id == Article.store_id)
    .filter(Price.timestamp >= start_date)
    .order_by(Store.id, desc(Price.price))
)

results = query.all().as_dict()

# display nicely in pandas
import pandas as pd
df = pd.DataFrame(results)
df.head(20)


2025-10-12 14:19:22,333 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-12 14:19:22,335 INFO sqlalchemy.engine.Engine SELECT stores.name AS store_name, articles.name AS article_name, prices.price AS prices_price, avg(prices.price) OVER (PARTITION BY stores.id) AS avg_store_price 
FROM prices JOIN articles ON articles.id = prices.article_id JOIN stores ON stores.id = articles.store_id 
WHERE prices.timestamp >= %(timestamp_1)s ORDER BY stores.id, prices.price DESC
2025-10-12 14:19:22,338 INFO sqlalchemy.engine.Engine [cached since 112.8s ago] {'timestamp_1': datetime.date(2025, 9, 12)}


AttributeError: 'list' object has no attribute 'as_dict'

In [3]:
from src.database_config.database import engine
from src.models.prices_latest import PriceLatest
from sqlalchemy.orm import Session

session = Session(engine)


# Query the row
x = session.query(PriceLatest).filter_by(article_id=100).first()

# Nice print
if x:
    print(f"PriceLatest[article_id={x.article_id}]: id={x.id}, price={x.price}, price_per_unit={x.price_per_unit}, timestamp={x.timestamp}")
else:
    print("No PriceLatest found for article_id=100")


2025-10-11 15:42:05,080 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 15:42:05,082 INFO sqlalchemy.engine.Engine SELECT prices_latest.id AS prices_latest_id, prices_latest.article_id AS prices_latest_article_id, prices_latest.price AS prices_latest_price, prices_latest.price_per_unit AS prices_latest_price_per_unit, prices_latest.price_per_unit_base AS prices_latest_price_per_unit_base, prices_latest.timestamp AS prices_latest_timestamp 
FROM prices_latest 
WHERE prices_latest.article_id = %(article_id_1)s 
 LIMIT %(param_1)s
2025-10-11 15:42:05,082 INFO sqlalchemy.engine.Engine [cached since 38.84s ago] {'article_id_1': 100, 'param_1': 1}
PriceLatest[article_id=100]: id=11256, price=3.79, price_per_unit=0.32, timestamp=2025-10-10


In [13]:
if (
        latest_price := session.query(PriceLatest)
        .filter_by(article_id=article.id)
        .first()
) and (
    latest_price.price == 2.39
) and (
    latest_price.price_per_unit == 0.24
    ):
        print(
            f"Unchanged price for article: {3}."
        )
else: 
    if not latest_price:
        latest_price = PriceLatest(article=article)
    print(
        f"New prices for article: {3}."
    )

NameError: name 'article' is not defined

In [12]:
from sqlalchemy import func, desc
from datetime import date, timedelta
from sqlalchemy.orm import joinedload
from src.models.articles import Article
from src.models.stores import Store
from src.models.prices import Price
from src.database_config.database import engine

from sqlalchemy.orm import Session

session = Session(engine)
articles = Article.query.session(session).all().as_dict()

NoSessionError: Cant get session.Please, call SaActiveRecord.set_session()

In [17]:
from src.models.stores import Store
from src.models.articles import Article
from src.models.prices import Price 
from src.models.prices_latest import PriceLatest 
from src.models.categories import Category 
from src.models.images import Image
from src.models.base import Base

from src.database_config.database import session 

from sqlalchemy import (
    func, desc
)
from sqlalchemy.orm import joinedload

articles = (
    Article.query.with_session(session)
    .options(
        joinedload(Article.store),
        joinedload(Article.price_latest)
    )
    .all()
)

# Convert to list of dicts automatically
data = articles  # works because these are ORM objects


2025-10-12 14:39:17,750 INFO sqlalchemy.engine.Engine SELECT articles.id AS articles_id, articles.ean_13 AS articles_ean_13, articles.name AS articles_name, articles.brand_name AS articles_brand_name, articles.invoice_unit AS articles_invoice_unit, articles.invoice_unit_type AS articles_invoice_unit_type, articles.store_id AS articles_store_id, stores_1.id AS stores_1_id, stores_1.name AS stores_1_name, prices_latest_1.id AS prices_latest_1_id, prices_latest_1.article_id AS prices_latest_1_article_id, prices_latest_1.price AS prices_latest_1_price, prices_latest_1.price_per_unit AS prices_latest_1_price_per_unit, prices_latest_1.price_per_unit_base AS prices_latest_1_price_per_unit_base, prices_latest_1.timestamp AS prices_latest_1_timestamp 
FROM articles LEFT OUTER JOIN stores AS stores_1 ON stores_1.id = articles.store_id LEFT OUTER JOIN prices_latest AS prices_latest_1 ON articles.id = prices_latest_1.article_id
2025-10-12 14:39:17,751 INFO sqlalchemy.engine.Engine [cached since 25

In [9]:
from src.models.base import Base 
Base.set_session(session=None)
